## Load and Preprocess

In [18]:
import tqdm
import random
import pathlib
import itertools
import collections

import cv2
import einops
import numpy as np
import remotezip as rz
import seaborn as sns
import matplotlib.pyplot as plt

import tensorflow as tf
import keras
from   keras import layers

In [ ]:
def list_files_per_class(zip_url):
    """
    List the files in each class of the dataset given the zip URL.
    @param zip_url: URL from which the files can be unzipped.
    @return files: List of files in each of the classes.
    """
    files = []
    with rz.RemoteZip(zip_url) as zip:
        for zip_info in zip.infolist():
            files.append(zip_info.filename)
        return files

def get_class(fname):
    """
    Retrieve the name of the class given a filename.
    @param fname: Name of the file in the UCF101 dataset.
    @return Class that the file belongs to.
    """
    return fname.split('_')[-3]

def get_files_per_class(files):
    """
    Retrieve the files that belong to each class.
    @param List of files in the dataset.
    @return Dictionary of class names (key) and files (values).
    """
    files_for_class = collections.defaultdict(list)
    for fname in files:
        class_name = get_class(fname)
        files_for_class[class_name].append(fname)
    return files_for_class

def download_from_zip(zip_url, to_dir, file_names):
    """
    Download the contents of the zip file from the zip URL.
    @param zip_url: Zip URL containing data.
    @param to_dir: Directory to download data to.
    @param file_names: Names of files to download.
    """
    with rz.RemoteZip(zip_url) as zip:
        for fn in tqdm.tqdm(file_names):
            class_name = get_class(fn)
            zip.extract(fn, str(to_dir / class_name))
            unzipped_file = to_dir / class_name / fn

            fn = pathlib.Path(fn).parts[-1]
            output_file = to_dir / class_name / fn
            unzipped_file.rename(output_file,)

def split_class_lists(files_for_class, count):
    """
    Returns the list of files belonging to a subset of data as well as the remainder of files that need to be downloaded.
    @param files_for_class: Files belonging to a particular class of data.
    @param Number of files to download.
    @return split_files: Files belonging to the subset of data.
    @return remainder: Dictionary of the remainder of files that need to be downloaded.
    """
    split_files = []
    remainder = {}
    for cls in files_for_class:
        split_files.extend(files_for_class[cls][:count])
        remainder[cls] = files_for_class[cls][count:]
    return split_files, remainder

def download_ufc_101_subset(zip_url, num_classes, splits, download_dir):
    """
    Download a subset of the UFC101 dataset and split them into various parts, such as training, validation, and test.
    @param zip_url: Zip URL containing data.
    @param num_classes: Number of labels.
    @param splits: Dictionary specifying the training, validation, test, etc. (key) division of data (value is number of files per split).
    @param download_dir: Directory to download data to.
    @return dir: Posix path of the resulting directories containing the splits of data.
    """
    files = list_files_per_class(zip_url)
    for f in files:
        tokens = f.split('/')
        if len(tokens) <= 2:
            files.remove(f) # Remove that item from the list if it does not have a filename

    files_for_class = get_files_per_class(files)

    classes = list(files_for_class.keys())[:num_classes]

    for cls in classes:
        new_files_for_class = files_for_class[cls]
        random.shuffle(new_files_for_class)
        files_for_class[cls] = new_files_for_class

    # Only use the number of classes you want in the dictionary
    files_for_class = {x: files_for_class[x] for x in list(files_for_class)[:num_classes]}

    dirs = {}
    for split_name, split_count in splits.items():
        print(split_name, ":")
        split_dir = download_dir / split_name
        split_files, files_for_class = split_class_lists(files_for_class, split_count)
        download_from_zip(zip_url, split_dir, split_files)
        dirs[split_name] = split_dir

    return dirs

def format_frames(frame, output_size):
    """
    Pad and resize an image from a video.
    @param frame: Image that needs to resized and padded.
    @param output_size: Pixel size of the output frame image.
    @return Formatted frame with padding of specified output size.
    """
    frame = tf.image.convert_image_dtype(frame, tf.float32)
    frame = tf.image.resize_with_pad(frame, *output_size)
    return frame

def frames_from_video_file(video_path, n_frames, output_size = (224,224), frame_step = 15):
    """
    Creates frames from each video file present for each category.
    @param video_path: File path to the video.
    @param n_frames: Number of frames to be created per video file.
    @param output_size: Pixel size of the output frame image.
    @return An NumPy array of frames in the shape of (n_frames, height, width, channels).
    """
    # Read each video frame by frame
    result = []
    src = cv2.VideoCapture(str(video_path))  

    video_length = src.get(cv2.CAP_PROP_FRAME_COUNT)

    need_length = 1 + (n_frames - 1) * frame_step

    if need_length > video_length:
        start = 0
    else:
        max_start = video_length - need_length
        start = random.randint(0, max_start + 1)

    src.set(cv2.CAP_PROP_POS_FRAMES, start)
    # ret is a boolean indicating whether read was successful, frame is the image itself
    ret, frame = src.read()
    result.append(format_frames(frame, output_size))

    for _ in range(n_frames - 1):
        for _ in range(frame_step):
            ret, frame = src.read()
            if ret:
                frame = format_frames(frame, output_size)
                result.append(frame)
            else:
                result.append(np.zeros_like(result[0]))
    src.release()
    result = np.array(result)[..., [2, 1, 0]]

    return result

In [27]:
# Generates frames
class FrameGenerator:
    def __init__(self, path, n_frames, training = False):
        """
        Returns a set of frames with their associated label.
        @param path: Video file paths.
        @param n_frames: Number of frames.
        @param training: Boolean to determine if training dataset is being created.
        """
        self.path = path
        self.n_frames = n_frames
        self.training = training
        self.class_names = sorted(set(p.name for p in self.path.iterdir() if p.is_dir()))
        self.class_ids_for_name = dict((name, idx) for idx, name in enumerate(self.class_names))

    def get_files_and_class_names(self):
        video_paths = list(self.path.glob('*/*.avi'))
        classes = [p.parent.name for p in video_paths] 
        return video_paths, classes

    def __call__(self):
        video_paths, classes = self.get_files_and_class_names()

        pairs = list(zip(video_paths, classes))

        if self.training:
            random.shuffle(pairs)

        for path, name in pairs:
            video_frames = frames_from_video_file(path, self.n_frames) 
            label = self.class_ids_for_name[name] # Encode labels
            yield video_frames, label

## Create the Model

In [19]:
class Conv2Plus1D(keras.layers.Layer):
    def __init__(self, filters, kernel_size, padding):
        """
        A sequence of convolutional layers that first apply the convolution operation over the spatial dimensions, and then the temporal dimension. 
        """
        super().__init__()
        self.seq = keras.Sequential([  
            # Spatial decomposition
            layers.Conv3D(filters = filters,
                            kernel_size = (1, kernel_size[1], kernel_size[2]),
                            padding = padding),

            # Temporal decomposition
            layers.Conv3D(filters = filters, 
                            kernel_size = (kernel_size[0], 1, 1),
                            padding = padding)
            ])

    def call(self, x):
        return self.seq(x)

In [20]:
class ResidualMain(keras.layers.Layer):
    """
    Residual block of the model with convolution, layer normalization, and the activation function, ReLU.
    """
    def __init__(self, filters, kernel_size):
        super().__init__()
        self.seq = keras.Sequential([
            Conv2Plus1D(filters = filters,
                        kernel_size = kernel_size,
                        padding = 'same'),
            layers.LayerNormalization(),
            layers.ReLU(),
            Conv2Plus1D(filters = filters, 
                        kernel_size = kernel_size,
                        padding = 'same'),
            layers.LayerNormalization()
        ])

    def call(self, x):
        return self.seq(x)

In [21]:
class Project(keras.layers.Layer):
    """
    Project certain dimensions of the tensor as the data is passed through different sized filters and downsampled. 
    """
    def __init__(self, units):
        super().__init__()
        self.seq = keras.Sequential([
            layers.Dense(units),
            layers.LayerNormalization()
        ])

    def call(self, x):
        return self.seq(x)

In [22]:
def add_residual_block(input, filters, kernel_size):
    """
    Add residual blocks to the model. If the last dimensions of the input data and filter size does not match, project it such that last dimension matches.
    """
    out = ResidualMain(filters, kernel_size)(input)
    res = input

    # Using the Keras functional APIs, project the last dimension of the tensor to match the new filter size
    if out.shape[-1] != input.shape[-1]:
        res = Project(out.shape[-1])(res)

    return layers.add([res, out])

In [23]:
class ResizeVideo(keras.layers.Layer):
    def __init__(self, height, width):
        super().__init__()
        self.height = height
        self.width = width
        self.resizing_layer = layers.Resizing(self.height, self.width)

    def call(self, video):
        """
        Use the einops library to resize the tensor.
        @param video: Tensor representation of the video, in the form of a set of frames.
        @return A downsampled size of the video according to the new height and width it should be resized to.
        """
        # b stands for batch size, t stands for time, h stands for height, w stands for width, and c stands for the number of channels.
        old_shape = einops.parse_shape(video, 'b t h w c')
        images = einops.rearrange(video, 'b t h w c -> (b t) h w c')
        images = self.resizing_layer(images)
        videos = einops.rearrange(
        images, '(b t) h w c -> b t h w c',
        t = old_shape['t'])
        return videos

In [24]:
# Define the dimensions of one frame in the set of frames created
HEIGHT = 28
WIDTH  = 28

In [25]:
input_shape = (None, 10, HEIGHT, WIDTH, 3)
input = layers.Input(shape = (input_shape[1:]))
x = input

x = Conv2Plus1D(filters=16, kernel_size = (3, 7, 7), padding='same')(x)
x = layers.BatchNormalization()(x)
x = layers.ReLU()(x)
x = ResizeVideo(HEIGHT // 2, WIDTH // 2)(x)

# Block 1
x = add_residual_block(x, 16, (3, 3, 3))
x = ResizeVideo(HEIGHT // 4, WIDTH // 4)(x)

# Block 2
x = add_residual_block(x, 32, (3, 3, 3))
x = ResizeVideo(HEIGHT // 8, WIDTH // 8)(x)

# Block 3
x = add_residual_block(x, 64, (3, 3, 3))
x = ResizeVideo(HEIGHT // 16, WIDTH // 16)(x)

# Block 4
x = add_residual_block(x, 128, (3, 3, 3))

x = layers.GlobalAveragePooling3D()(x)
x = layers.Flatten()(x)
x = layers.Dense(10)(x)

model = keras.Model(input, x)

model.summary()

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 10, 28, 28,  0           []                               
                                 3)]                                                              
                                                                                                  
 conv2_plus1d_18 (Conv2Plus1D)  (None, 10, 28, 28,   3152        ['input_3[0][0]']                
                                16)                                                               
                                                                                                  
 batch_normalization_2 (BatchNo  (None, 10, 28, 28,   64         ['conv2_plus1d_18[0][0]']        
 rmalization)                   16)                                                         

## Compile and Fit the Model

In [26]:
# Defines the optimizer
customOptimizer = keras.optimizers.Adam(5e-4)

# Defines the loss function
customLoss = keras.losses.SparseCategoricalCrossentropy()

# Defines the callbacks to include
my_callbacks = [
    #tf.keras.callbacks.ModelCheckpoint(filepath = "./MNIST Models 2D/Accuracy/mnist_detection.acc.{epoch:02d}-{val_accuracy:.2f}.h5", monitor = "val_accuracy", mode = "max", save_best_only = True),
    #tf.keras.callbacks.ModelCheckpoint(filepath = "./MNIST Models 2D/Loss/mnist_detection.loss.{epoch:02d}-{val_loss:.2f}.h5", monitor = "val_loss", mode = "min", save_best_only = True),
    tf.keras.callbacks.EarlyStopping  (patience = 200, monitor = "accuracy"),
    #tf.keras.callbacks.TensorBoard    (log_dir  = "./Logs")
]

# Compile the model
model.compile(loss = customLoss, optimizer = customOptimizer, metrics = ['accuracy'])

In [ ]:
# Fits the model
history = model.fit(x = train_ds, epochs = 10, validation_data = val_ds, callbacks = my_callbacks, use_multiprocessing = True)